In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [150]:
Discrete(5,start = 95).sample()


98

In [225]:
Box(low=np.array([[-1.0, -2.0], [-1.0, -2.0]]),high=np.array([[2.0, 4.0], [2.0, 4.0]]), dtype=int).sample()


array([[-1, -1],
       [-1,  0]])

In [233]:
Box(0,255,shape=(720,1280), dtype=int).sample()


array([[ 78, 107,  32, ...,  96,  73,  21],
       [199, 226,  82, ..., 174, 229,  45],
       [101, 170, 210, ..., 190, 166,  45],
       ...,
       [ 53, 124, 240, ..., 163,  49, 105],
       [226,  72, 109, ..., 179, 135, 134],
       [ 44,  11, 189, ...,  51, 249, 232]])

In [252]:
Tuple((Discrete(2), Box(0,100, shape=(6,)))).sample()


(0,
 array([57.40844 , 65.890915, 30.453634,  1.536176, 83.35072 , 32.412567],
       dtype=float32))

In [254]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()


OrderedDict([('height', 0), ('speed', array([8.015858], dtype=float32))])

In [264]:
MultiBinary(4).sample()


array([0, 1, 0, 1], dtype=int8)

In [329]:
MultiDiscrete([255,1,2]).sample()


array([71,  0,  0], dtype=int64)

In [390]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

In [391]:
env=ShowerEnv()


In [389]:
env.observation_space.sample()


array([66.26255], dtype=float32)

In [387]:
env.reset()


array([39.])

In [392]:
from stable_baselines3.common.env_checker import check_env


In [404]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


Episode:1 Score:-32
Episode:2 Score:-12
Episode:3 Score:-42
Episode:4 Score:-8
Episode:5 Score:-58


In [405]:
log_path = os.path.join('Training', 'Logs')


In [406]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Yash\AppData\Roaming\Python\Python310\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [408]:

model.learn(total_timesteps=400000)

Logging to Training\Logs\PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -31.6    |
| time/              |          |
|    fps             | 2135     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -25.7        |
| time/                   |              |
|    fps                  | 1462         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0027850987 |
|    clip_fraction        | 0.0237       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.08        |
|    explained_variance   |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 10          |
| time/                   |             |
|    fps                  | 1163        |
|    iterations           | 11          |
|    time_elapsed         | 19          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.010493293 |
|    clip_fraction        | 0.19        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.961      |
|    explained_variance   | 0.0215      |
|    learning_rate        | 0.0003      |
|    loss                 | 21.6        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0127     |
|    value_loss           | 31.6        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 36.7        |
| time/                   |             |
|    fps                  | 1134        |
|    iterations           | 21          |
|    time_elapsed         | 37          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.023711506 |
|    clip_fraction        | 0.248       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.84       |
|    explained_variance   | -5.97e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 25.2        |
|    n_updates            | 300         |
|    policy_gradient_loss | 0.0125      |
|    value_loss           | 43.2        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 35.9        |
| time/                   |             |
|    fps                  | 1122        |
|    iterations           | 31          |
|    time_elapsed         | 56          |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.044333383 |
|    clip_fraction        | 0.3         |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.794      |
|    explained_variance   | -8.58e-06   |
|    learning_rate        | 0.0003      |
|    loss                 | 31.5        |
|    n_updates            | 400         |
|    policy_gradient_loss | 0.012       |
|    value_loss           | 55.5        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 40.2       |
| time/                   |            |
|    fps                  | 1118       |
|    iterations           | 41         |
|    time_elapsed         | 75         |
|    total_timesteps      | 83968      |
| train/                  |            |
|    approx_kl            | 0.02587017 |
|    clip_fraction        | 0.265      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.831     |
|    explained_variance   | -1.43e-06  |
|    learning_rate        | 0.0003     |
|    loss                 | 48.8       |
|    n_updates            | 500        |
|    policy_gradient_loss | 0.0162     |
|    value_loss           | 80.3       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_m

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 55.8        |
| time/                   |             |
|    fps                  | 1109        |
|    iterations           | 151         |
|    time_elapsed         | 278         |
|    total_timesteps      | 309248      |
| train/                  |             |
|    approx_kl            | 0.006967956 |
|    clip_fraction        | 0.161       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.653      |
|    explained_variance   | 1.14e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 78.2        |
|    n_updates            | 1600        |
|    policy_gradient_loss | 0.0137      |
|    value_loss           | 113         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 58          |
| time/                   |             |
|    fps                  | 1109        |
|    iterations           | 161         |
|    time_elapsed         | 297         |
|    total_timesteps      | 329728      |
| train/                  |             |
|    approx_kl            | 0.012501353 |
|    clip_fraction        | 0.172       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.68       |
|    explained_variance   | 8.76e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 61.8        |
|    n_updates            | 1700        |
|    policy_gradient_loss | 0.0122      |
|    value_loss           | 116         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 51.8        |
| time/                   |             |
|    fps                  | 1108        |
|    iterations           | 171         |
|    time_elapsed         | 315         |
|    total_timesteps      | 350208      |
| train/                  |             |
|    approx_kl            | 0.036165178 |
|    clip_fraction        | 0.121       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.579      |
|    explained_variance   | -0.000399   |
|    learning_rate        | 0.0003      |
|    loss                 | 49.2        |
|    n_updates            | 1800        |
|    policy_gradient_loss | 0.00962     |
|    value_loss           | 109         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 53          |
| time/                   |             |
|    fps                  | 1109        |
|    iterations           | 181         |
|    time_elapsed         | 334         |
|    total_timesteps      | 370688      |
| train/                  |             |
|    approx_kl            | 0.011681369 |
|    clip_fraction        | 0.128       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.57       |
|    explained_variance   | 0.000109    |
|    learning_rate        | 0.0003      |
|    loss                 | 33.8        |
|    n_updates            | 1900        |
|    policy_gradient_loss | 0.00882     |
|    value_loss           | 99.7        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 58.4        |
| time/                   |             |
|    fps                  | 1108        |
|    iterations           | 191         |
|    time_elapsed         | 352         |
|    total_timesteps      | 391168      |
| train/                  |             |
|    approx_kl            | 0.016468134 |
|    clip_fraction        | 0.168       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.589      |
|    explained_variance   | -1.91e-06   |
|    learning_rate        | 0.0003      |
|    loss                 | 59.6        |
|    n_updates            | 2000        |
|    policy_gradient_loss | 0.0146      |
|    value_loss           | 118         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

In [410]:
model.save('Training\Saved Models\PPO')


In [416]:
evaluate_policy(model, env, n_eval_episodes=200, render=True)

(59.41, 0.9120855223058855)